<img src="worms.png"/>
<h1> WoRMS RestAPI Walkthrough</h1>
<p> Provided is a general outline of how the WoRMS Rest API can be accessed using python.</p>
<p>the Recommended version of python is <b>Python 3.7+</b> which can be downloaded from <a href="https://python.org/">this</a> link.<br>The required python libraries are listed in the accompanying environment.yml file</p>
<hr>
<h2> Python Imports </h2>
<p>Always run the cell below to initialize your python running environment. The requests package will be used to make calls to the WoRMS API and returned will be json<br>
    More information about requests can be obtained <a href="https://requests.readthedocs.io/">here</a></p>

In [ ]:
# Import requests and set the WoRMS API base URL. 
import requests
import json

# Convenience function to pretty print json objects and python dicts
def print_json(myjson):
    print(json.dumps(
        myjson,
        sort_keys=True,
        indent=4,
        separators=(',', ': ')
    ))

# Initialize the base URL for WoRMS. This variable is unchanged for every api call 
WORMS_URL = "http://www.marinespecies.org/rest"

## Getting AphiaIDs
API calls to retrieve aphiaIDs from the WoRMS system. AphiaIDs are the IDs that link to a specific species in the WoRMS database.

## Search for AphiaIDs using vernacular
Basic search using the WoRMS API to get back a AphiaID from a vernacular name 

In [ ]:
# Lets search WoRMS to get the aphiaID using Atlantic salmon as the vernacular input
vernacular = 'Atlantic Salmon'

# AphiaRecordsByVernacular
req = requests.get(f'{WORMS_URL}/AphiaRecordsByVernacular/{vernacular}')
print_json(req.json())

### Search using Multilingual vernacular names?
WoRMS accepts vernacular names in a multitude of difference languages as shown below.

In [ ]:
# Try in french and see if we get the same aphiaID for 'Saumon atlantique'
vernacular = 'Saumon atlantique'

# AphiaRecordsByVernacular
req = requests.get(f'{WORMS_URL}/AphiaRecordsByVernacular/{vernacular}')
print_json(req.json())

### Searching aphiaID by using scientific name
Use the API call AphiaRecordsByName to retrieve aphiaID WoRMS records using the scientific name

In [ ]:
# Now let's try the scientific name for Atlantic Salmon, 'Salmo salar'
scientificName = 'Salmo Salar'

# AphiaRecordsByName
req = requests.get(f'{WORMS_URL}/AphiaRecordsByName/{scientificName}')
print_json(req.json())

### Refine AphiaRecordsByName search

In [ ]:
# The above cell returned more results than needed. Since we know the exact scientific name
# we are going to add "like=false" to the query,
# this will exclude all the extra like matched results.
scientificName = 'Salmo Salar'
like_matched = 'false'  

# AphiaRecordsByName
req = requests.get(f'{WORMS_URL}/AphiaRecordsByName/{scientificName}?like={like_matched}')
print_json(req.json())

## Species Attributes
Using a valid aphiaID, retrieve species specific attribute data held in WoRMs

In [ ]:
# Now that we got the valid aphiaID for our Atlatic Salmon species, lets use WoRMS to retrieve some species attributes.
aphiaID = 127186 # Atlantic Salmon ID
req = requests.get(f'{WORMS_URL}/AphiaAttributesByAphiaID/{aphiaID}')
attributes = req.json()

print_json(attributes)

In [ ]:
# Whoa, that was a bunch of attribute data. 

# The standard columns for measurements are:
['AphiaID', 'measurementTypeID', 'measurementType', 'measurementValue', 'source_id', 
 'reference', 'qualitystatus', 'AphiaID_Inherited', 'CategoryID', 'children']

# Extra measurement data is being held in the children column item so 
# joining the all children nodes will yeild more visually uncomplicated records
repeated_columns = ['AphiaID','reference','qualitystatus','AphiaID_Inherited','source_id']

# recursive children walking function
def walk(nodes, row, level=1):
    for node in nodes:
        for k in node.keys():
            if k == 'children':
                walk(node[k], row)
            elif k not in repeated_columns:
                row[str(node['measurementTypeID']) + '_' + k] = node[k]
          
# Loop through all the Atlantic Salmon attributes
for attr in attributes:
    row = {}
    for k in attr.keys():
        if k == 'children':
            walk(attr[k], row)
        else:
            row[k] = attr[k]
            
    display(row)
    print('-' * 120)

### Get more info about measurementTypes accepted for a CategoryID
The call to AphiaAttributeValuesByCategoryID will show all availible acceptable values for a given CategoryID

In [ ]:
# Given an measurementTypeID IUCN Red List Category (1), quering it's categoryID (1) 
# will show all the accepted values for this category

CategoryID = 1
# AphiaAttributeValuesByCategoryID
req = requests.get(f'{WORMS_URL}/AphiaAttributeValuesByCategoryID/{CategoryID}')
req.json()

## Distributions
Querying for distributions using AphiaDistributionsByAphiaID will return regions where a species has been observed including marineregions.org links. 

In [ ]:
# Lets retrieve the list of distributions on record for Atlantic Salmon, with record status and quality status
req = requests.get(WORMS_URL + f'/AphiaDistributionsByAphiaID/{aphiaID}')
distributions = req.json()

for distrib in distributions:
    locality = distrib['locality']
    location_id = distrib['locationID']
    record_status = distrib['recordStatus']
    quality_status = distrib['qualityStatus']
    print(locality, location_id, record_status, quality_status)

In [ ]:
# On to mapping the distributions for Atlantic Salmon. We will accomplish this using folium and the links 
# WoRMS gives to MarineRegions.org
import folium
m = folium.Map() # Create a map object

for distrib in distributions:
    locality = distrib['locality']
    location_id = distrib['locationID']
    
    # retrieve location info from MarineRegion's API
    req = requests.get(location_id)
    location = req.json()
    max_lat = location['maxLatitude']
    min_lat = location['minLatitude']
    max_lon = location['maxLongitude']
    min_lon = location['minLongitude']
    
    # If all values are present, create bounding box, otherwise create point
    if all([max_lat, min_lat, max_lon, min_lon]):
        # Creating the polygon in a clockwise pattern
        bbox = folium.Polygon([[min_lat, min_lon],
                               [max_lat, min_lon],
                               [max_lat, max_lon],
                               [min_lat, max_lon],
                               [min_lat, min_lon]], popup=location)
        m.add_child(bbox) # add bounding box to map 
    else:
        point = folium.Marker([location['latitude'], location['longitude']], popup=location)
        m.add_child(point) # add marker points to map
    
m # Display the resulting map with distribtions shown

## External Indentifiers
Many other Taxonomic databases use their own ID systems to reference species records. Using AphiaExternalIDByAphiaID for a given aphiaID, you can retrieve these external IDs as they are known to WoRMS.

In [ ]:
# Let's get all the external IDs for Atlantic Salmon from external taxonomic systems known to WoRMS

external_ids = {'algaebase': 'Algaebase species ID',
    'bold': 'Barcode of Life Database (BOLD) TaxID',
    'dyntaxa': 'Dyntaxa ID',
    'eol': 'Encyclopedia of Life (EoL) page identifier',
    'fishbase': 'FishBase species ID',
    'iucn': 'IUCN Red List Identifier',
    'lsid': 'Life Science Identifier',
    'ncbi':' NCBI Taxonomy ID (Genbank)',
    'tsn': 'ITIS Taxonomic Serial Number',
    'gisd': 'Global Invasive Species Database'}

# Loop and print out all known IDs
for ext in external_ids.keys():
    # AphiaExternalIDByAphiaID
    req = requests.get(f'{WORMS_URL}/AphiaExternalIDByAphiaID/{aphiaID}?type={ext}')
    print(f'{ext}: ', end="")
    if req.content:
        print(req.json()[0])
    else:
        print('None')

## Sources
Get source and reference data given a valid aphiaID value.

In [ ]:
# Now we are going to retrieve all the source reference data Atlantic Salmon
req = requests.get(f'{WORMS_URL}/AphiaSourcesByAphiaID/{aphiaID}')
req.json()

## Taxonomic data
Directly query taxonomic data. Source/definitions borrowed from http://www.marinespecies.org/rest/

In [ ]:
# AphiaChildrenByAphiaID
req = requests.get(f'{WORMS_URL}/AphiaChildrenByAphiaID/{aphiaID}')
req.json()

In [ ]:
# the direct children (max. 50) for a given AphiaID
    
# AphiaChildrenByAphiaID
req = requests.get(f'{WORMS_URL}/AphiaChildrenByAphiaID/{aphiaID}')
req.json()

In [ ]:
# the complete classification for one taxon. This also includes any sub or super ranks.
    
# AphiaClassificationByAphiaID
req = requests.get(f'{WORMS_URL}/AphiaClassificationByAphiaID/{aphiaID}')
req.json()

In [ ]:
# the AphiaID for a given name

scientific_name = 'Salmo salar'
# AphiaIDByName
req = requests.get(f'{WORMS_URL}/AphiaIDByName/{scientific_name}')
req.json()

In [ ]:
# the name for a given AphiaID
    
# AphiaNameByAphiaID
req = requests.get(f'{WORMS_URL}/AphiaNameByAphiaID/{aphiaID}')
req.json()

In [ ]:
# the complete AphiaRecord for a given AphiaID
    
# AphiaRecordByAphiaID
req = requests.get(f'{WORMS_URL}/AphiaRecordByAphiaID/{aphiaID}')
req.json()

In [ ]:
# an AphiaRecord for multiple AphiaIDs in one go (max: 50)

aphiaID1 = 127186
aphiaID2 = 137116

# AphiaRecordsByAphiaIDs
req = requests.get(f'{WORMS_URL}/AphiaRecordsByAphiaIDs?aphiaids[]={aphiaID1}&aphiaids[]={aphiaID2}')
req.json()

In [ ]:
# Lists all AphiaRecords (taxa) that have their last edit action (modified or added) during the specified period

startdate='1998-10-01'
enddate='1999-12-31'
# AphiaRecordsByDate
req = requests.get(f'{WORMS_URL}/AphiaRecordsByDate?startdate={startdate}&enddate={enddate}')
req.json()

In [ ]:
# Try to find AphiaRecords using the TAXAMATCH fuzzy matching algorithm by Tony Rees

scientific_name = 'Salmo salarr'
# AphiaRecordsByMatchNames
req = requests.get(f'{WORMS_URL}/AphiaRecordsByMatchNames?scientificnames[]={scientific_name}')
req.json()

In [ ]:
# one or more matching (max. 50) AphiaRecords for a given name

scientific_name = 'Salmo salar'

# AphiaRecordsByName
req = requests.get(f'{WORMS_URL}/AphiaRecordsByName/{scientific_name}')
req.json()

In [ ]:
# For each given scientific name, try to find one or more AphiaRecords

scientific_name1 = 'Salmo salar'
scientific_name2 =  'Monodon monoceros'
# AphiaRecordsByNames
req = requests.get(f'{WORMS_URL}/AphiaRecordsByNames?scientificnames[]={scientific_name1}'
                   f'&scientificnames[]={scientific_name2}')
req.json()

In [ ]:
# the AphiaRecords for a given taxonRankID (max 50)
    
rankID = '220' # Species
# AphiaRecordsByTaxonRankID
req = requests.get(f'{WORMS_URL}/AphiaRecordsByTaxonRankID/{rankID}')
req.json()

In [ ]:
# all synonyms for a given AphiaID
    
# AphiaSynonymsByAphiaID
req = requests.get(f'{WORMS_URL}/AphiaSynonymsByAphiaID/{aphiaID}')
req.json()

In [ ]:
# taxonomic ranks by their identifier

rankID = '220' # Species
# AphiaTaxonRanksByID
req = requests.get(f'{WORMS_URL}/AphiaTaxonRanksByID/{rankID}')
req.json()

In [ ]:
# taxonomic ranks by their name
    
# AphiaTaxonRanksByName
req = requests.get(f'{WORMS_URL}/AphiaTaxonRanksByName/{aphiaID}')
req.json()